In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl evaluate
!pip install git+https://github.com/gpassero/uol-redacoes-xml.git

In [ ]:
import torch
import nltk
nltk.download('punkt_tab')
import uol_redacoes_xml
import pandas as pd
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
essays = uol_redacoes_xml.load()
data = []

for essay in essays:
  if essay.comments.find('<comments>') or essay.comments.find('</comments>'):
    essay.comments = essay.comments.replace('<comments>', '')
    essay.comments = essay.comments.replace('</comments>', '')

  data.append({
      'THEME': essay.prompt.title,
      'ESSAY': essay.text,
      'COMMENTS': essay.comments
  })

filename = 'Essays.csv'
df_essays = pd.DataFrame(data)
df_essays.drop_duplicates(subset='REDACAO', inplace=True)
df_essays.dropna(inplace=True)
df_essays.to_csv(filename, index=False)

In [ ]:
model_id = "amadeusai/AV-FI-Qwen2.5-0.5B-PT-BR-Instruct"

# Tokenização do modelo pré-treinado
tok = AutoTokenizer.from_pretrained(model_id)
# Questão de segurança para modelos criptografados
if tok.pad_token is None: tok.pad_token = tok.eos_token

# Carregamento do modelo
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="auto",
    trust_remote_code=True # Permite o uso de modelos com arquiteturas mais recentes que a biblioteca não conhece
)
model

In [ ]:
lora_cfg = LoraConfig(
    r=8, # Maior capacidade de aprender detalhes complexos devido a um número maior de parâmetros
    lora_alpha=16, # Força do impacto do que o LoRA aprendeu sobre o conhecimento original do modelo (valor alto = novo treino se sobrepõe ao modelo)
    lora_dropout=0.05, # A cada passo do treino, 5% dos neurônios do LoRA são "desligados"
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# Envolve o modelo base com o LoRA
model = get_peft_model(model, lora_cfg)
model

In [ ]:
# Template para prompt
IN_FMT = """Corrija a redação do ENEM.\n\nTema: {tema}\n\nRedação:\n{redacao}\n\nFeedback:\n"""

# Conversão do DataFrame para um dataset do tipo Hugging Face
ds = Dataset.from_pandas(df_essays)
ds

In [ ]:
# Tamanho máximo da sequência
MAX_LEN = 512

def tok_ex(ex):
  # Formatação da parte de instrução do prompt
  prompt = IN_FMT.format(tema=ex['THEME'], redacao=ex['ESSAY'])
  # Converte o texto do prompt em tokens sem adição de marcadores de início e fim de frase
  tp = tok(prompt, add_special_tokens=False)
  # Converte o texto da resposta em tokens sem adição de marcadores de início e fim de frase
  tr = tok(ex['COMMENTS'] + tok.eos_token, add_special_tokens=False)
  # Concatenar tokens de prompt e de resposta
  ids = (tp['input_ids'] + tr['input_ids'])[:MAX_LEN]
  # Criação da máscara de atenção
  att = [1] * len(ids)
  # Criação dos labels (modelo não deve treinar no prompt, apenas na resposta)
  lab = ([-100]*len(tp["input_ids"]) + tr["input_ids"])[:MAX_LEN] # -100 no prompt: o cálculo de loss dessa parte é ignorado
  # Exemplo tokenizado com os IDs de entrada, máscara de atenção e labels
  return {"input_ids": ids, "attention_mask": att, "labels": lab}

train_ds = ds.map(tok_ex, remove_columns=ds.column_names)
train_ds

In [ ]:
# batch: lista com exemplos do dataset
def collate(batch):
  # Função auxiliar para auxiliar na organização das redações
  def pad_list(lst, pad_val):
    # Maior texto
    maxlen = max(len(x) for x in lst)

    out = []
    # Com os textos menores, adiciona um padding ao final até que fiquem do tamanho do maior
    for x in lst:
      # Conversão para um tensor
      t = torch.tensor(x, dtype=torch.long)
      # Adição do padding
      if t.size(0) < maxlen:
        t = torch.nn.functional.pad(t, (0, maxlen - t.size(0)), value=pad_val)
      out.append(t)

    # Transforma a lista em uma matriz (tensor)
    return torch.stack(out)

  # Extrai os input IDs de todos os exemplos no batch
  ids = [b["input_ids"] for b in batch]
  # Extrai as máscaras de atenção de todos os exemplos no batch
  att = [b["attention_mask"] for b in batch]
  # Extrai os labels de todos os exemplos no batch
  lab = [b["labels"] for b in batch]

  # Texto (input_ids)
  # Preenche o espaço vazio com um token nulo para o modelo saber que aquilo não é uma palavra

  # Atenção (attention_mask)
  # 0 para ignorar, caso contrário, o modelo tentará ler um espaço em branco

  # Gabarito (labels)
  # -100 para ignorar, assim o modelo não é avaliado por acertar um "espaço em branco"
  return{
      "input_ids": pad_list(ids, tok.pad_token_id),
      "attention_mask": pad_list(att, 0),
      "labels": pad_list(lab, -100)
  }

In [ ]:
args = TrainingArguments(
    output_dir="./redacao",
    num_train_epochs=2, # Número de épocas = quantidade de vezes que o dataset é apresentado para o modelo
    per_device_train_batch_size=2, # Tamanho da lista (batch) por GPU
    gradient_accumulation_steps=4, # Batchs efetivos = 16
    learning_rate=2e-4, # Taxa de aprendizado = velocidade que o modelo aprende
    logging_steps=10, # Frequência com que o treinamento imprime os resultados
    save_steps=50,
    report_to="none", # Desativa o envio de relatórios para ferramentas externas
    bf16=True, # GPU otimizada para trabalhar com 16 bits, acelerando cálculos
    optim="paged_adamw_32bit", # Evita que o treino seja interrompido for falta de recursos (quando a memória chega no limite)
    group_by_length=True # Organiza os batch para que textos de tamanhos parecidos fiquem juntos
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collate)
trainer.model.config

In [ ]:
trainer.train()

In [ ]:
def avaliar_geracao(original, generated):
  reference = [word_tokenize(original, language='portuguese')]
  candidate = word_tokenize(generated, language='portuguese')

  score = sentence_bleu(reference, candidate)
  return score

In [ ]:
def generate(instr):
  prompt = IN_FMT.format(tema=instr['THEME'], redacao=instr['ESSAY'])
  x = tok(prompt, return_tensors="pt").to(model.device)

  with torch.no_grad():
    y = model.generate(
        **x,
        max_new_tokens=1024,
        do_sample=False,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
        no_repeat_ngram_size=3
    )

  # Calculamos o tamanho do prompt original para cortá-lo da resposta
  input_len = x["input_ids"].shape[1]
  # Pegamos apenas os tokens que vieram DEPOIS do prompt
  generated_tokens = y[0][input_len:]
  # Decodificamos apenas a resposta nova
  generated_text = tok.decode(generated_tokens, skip_special_tokens=True)
  # Pegamos o gabarito original
  original_text = instr['COMMENTS']

  score = avaliar_geracao(original_text, generated_text)
  print("="*40)
  print(f"TEMA: {instr['TEMA']}")
  print("-" * 40)
  print("SAÍDA DO MODELO (Gerado):")
  print(generated_text)
  print("-" * 40)
  print("GABARITO (Esperado):")
  print(original_text)
  print("="*40)
  print(f">> BLEU Score: {score:.3f}")

  return score

In [ ]:
exemplo = df_redacoes.iloc[5]
score = generate(exemplo)